# Nootebook to calculate the coordinates of the dla

In [5]:
import h5py
import numpy as np
from matplotlib import pyplot as plt

### Read the data from both files

In [6]:
# specify the path to the two files with Lyman alpha skewers, with (ON) and without (OFF) self-shielding 
fname_off='/data/desi/common/astrid/spectra_ASTRID_self-shield_off_z2.5_500x500x2500.hdf5'
fname_on='/data/desi/common/astrid/spectra_ASTRID_z2.5_500x500x2500.hdf5'

In [7]:
with h5py.File(fname_on,'r') as f:
    tau_on = f['tau/H/1/1215'][:]
    colden = f['colden']['H/1'][:]
    axes = f['spectra/axes'][:]
    # (x,y,z) start of the skewers, in kpc/h
    cofm_hkpc = f['spectra/cofm'][:]

In [9]:
with h5py.File(fname_off,'r') as f:
    tau_off = f['tau/H/1/1215'][:]
    # column densities were not stored for this file (should be the same as in the other file)
    #colden_off = f['colden']['H/1'][:]
    axes_off = f['spectra/axes'][:]
    cofm_off = f['spectra/cofm'][:]

### Define the functions to find the skewers

In [10]:
# box lenght in Mpc/h
# this information should be stored in the HDF5, but I couldn't find it... so added it by hand
L_hMpc=250
# number of skewers per side (500)
Nsk=int(np.sqrt(colden.shape[0]))
print(Nsk,'skewers per side')
# number of pixels / cells per skewer (2500)
Np=int(colden.shape[1])
print(Np,'pixels per skewer')
# pixel width (in Mpc/h)
dz_hMpc=L_hMpc/Np
print('pixel width = {:.3f} Mpc/h'.format(dz_hMpc))
# minimum separation between skewers (in Mpc/h)
dxy_hMpc=L_hMpc/Nsk
print('skewer separation = {:.3f} Mpc/h'.format(dxy_hMpc))

500 skewers per side
2500 pixels per skewer
pixel width = 0.100 Mpc/h
skewer separation = 0.500 Mpc/h


In [11]:
def skewer_index(ix, iy):
    """Given the ix, iy indices, return the global index of a given skewer"""
    return ix*Nsk+iy

In [12]:
def skewer_ix_iy(isk):
    """Given the global index, return the (ix, iy) indices of a skewer"""
    ix=isk//Nsk
    iy=isk-ix*Nsk    
    return ix, iy

In [13]:
def skewer_xy_hMpc(isk):
    """Return the (x,y) coordinates of a given skewer, in Mpc/h"""
    ix, iy = skewer_ix_iy(isk)
    return dxy_hMpc*ix, dxy_hMpc*iy

In [16]:
'''
coords = []
for i in range(0,501):
    x, y = skewer_xy_hMpc(i)
    max_colden=np.max(colden.reshape([Nsk,Nsk,Np]),axis=2)
    plot_mask = ((max_colden > 1e19) & (max_colden < 1e20)).flatten()
    plot_isk=np.where(plot_mask)[0][0]
    idla=np.argmax(colden[plot_isk])
    z = np.argmax(colden[plot_isk])
    coords.append([x, y, z])
print(coords)
'''

'\ncoords = []\nfor i in range(0,501):\n    x, y = skewer_xy_hMpc(i)\n    max_colden=np.max(colden.reshape([Nsk,Nsk,Np]),axis=2)\n    plot_mask = ((max_colden > 1e19) & (max_colden < 1e20)).flatten()\n    plot_isk=np.where(plot_mask)[0][0]\n    idla=np.argmax(colden[plot_isk])\n    z = np.argmax(colden[plot_isk])\n    coords.append([x, y, z])\nprint(coords)\n'